In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

In [69]:
scaler=pickle.load(P('问题2-MinMaxScaler-训练数据.csv').open('rb'))
# df=pd.read_csv('问题2-训练数据-归一化.csv',index_col='time', parse_dates=True)
df=pd.read_csv('问题2-训练数据-归一化.csv').drop(columns='time')
model=keras.models.load_model('./problem2_checkpoint.h5')

In [70]:
df_test = df.iloc[-3:, :]
test=df_test.values.reshape([1,3,-1])
df_test,test

(     10cm湿度(kg/m2)  40cm湿度(kg/m2)  100cm湿度(kg/m2)  土壤蒸发量(mm)   降水量(mm)  \
 120       0.225522       0.701157        1.000000   0.004360  0.681001   
 121       0.197432       0.701157        1.000000   0.000000  0.892632   
 122       0.426966       0.700844        0.999804   0.146448  0.129593   
 
       平均气温(℃)  
 120  0.129112  
 121  0.111732  
 122  0.433892  ,
 array([[[0.22552167, 0.70115661, 1.        , 0.00436009, 0.68100113,
          0.12911235],
         [0.19743178, 0.70115661, 1.        , 0.        , 0.89263167,
          0.11173184],
         [0.42696629, 0.70084401, 0.99980427, 0.14644781, 0.12959329,
          0.43389199]]]))

In [71]:
pred=model.predict(test)
df_pred=pd.DataFrame(pred, columns=df.columns)
df_pred

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,0.235638,0.563554,0.822945,0.046143,0.514566,0.195223


In [75]:
pd.concat([df_test, df_pred]).shift(1).dropna()

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
121,0.225522,0.701157,1.000000,0.004360,0.681001,0.129112
122,0.197432,0.701157,1.000000,0.000000,0.892632,0.111732
0,0.426966,0.700844,0.999804,0.146448,0.129593,0.433892


In [ ]:
def predict(model, init_inputs):
    